# Determining Partial EDX Ionization Cross Sections from a FIB Wedge of Known Angle

This is a personal workbook used to process my own data but has been made public for the use of others. If you find any errors or need further help using this workbook please contact k8macarthur@gmail.com.

## Author

Katherine E. MacArthur - Originally written for EMC2016, 28.08.2016

## Requirements

This code was written to work Hyperspy version 1.1 or later.

## 1. Importing Hyperspy and Libraries

Begin by importing the Hyperspy Function Library.

In [25]:
import matplotlib
%matplotlib nbagg
import hyperspy.api as hs
import numpy as np

For inline displaying of figures (optional).

In [2]:
%matplotlib inline

## 2. Importing the Wedge EDX Map for Element 1 (Pt in this Example)

Opens a load window. Use hs.load('filename') if filename is known specifically.

In [27]:
Pt_wedge = hs.load('EDX-Data/Pt-EDX04.bcf')

c:\users\macark\documents\github\hyperspy\hyperspy\io_plugins\bcf.py:829: RuntimeWarning: overflow encountered in long_scalars
  real_time = line_cnt_sum * line_avg * pix_avg * pix_time * width / 1000000.0


These next two lines are needed for .bcf files from Bruker/Espirit as the imported data comes as a list containing both the ADF image and EDX spectral data cube.

In [28]:
Pt_wedge_image = Pt_wedge[0]
Pt_wedge = Pt_wedge[1]

In [29]:
Pt_wedge.set_signal_type('EDS_TEM')
Pt_wedge.metadata.General.title = 'Original Data Wedge'

Check that the wedge has sensible dimensions and shape, 
that the labelling has been done correctly.

In [30]:
Pt_wedge

<EDSTEMSpectrum, title: Original Data Wedge, dimensions: (1024, 1024|4096)>

Adjust the metadata for all the microscope parameters.

In [31]:
Pt_wedge.axes_manager[0].name = 'x'
Pt_wedge.axes_manager[1].name = 'y'
Pt_wedge.axes_manager

<Axes manager, axes: (1024, 1024|4096)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |   1024 |      0 |       0 |   0.012 |     µm 
               y |   1024 |      0 |       0 |   0.012 |     µm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   4096 |        |   -0.95 |    0.01 |    keV

The axes_manager.gui() can be used to set all the axis properties. The scale of the navigational axis should be in nm for the later processing.

In [33]:
Pt_wedge.axes_manager.gui()

The next two lines can be used if the data needs cropping before further analyis. 

Data is cropped based on the pixel number ranges used. The crop range can be plotted first, before cropping.

In [39]:
Pt_wedge.inav[0:1024, 278:342].plot()

c:\program files\anaconda3\lib\site-packages\matplotlib\axes\_base.py:3193: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0, top=0
  'bottom=%s, top=%s') % (bottom, top))


Cropping is performed and then the new data replotted.

In [40]:
Pt_wedge0 = Pt_wedge
Pt_wedge = Pt_wedge.inav[0:1024, 278:342]
Pt_wedge.plot()

In [41]:
Pt_wedge.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 45.0
│       │       ├── detector_type = SuperX
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       └── real_time = -1345.709056
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       ├── beam_energy = 200
│       └── magnification = 7100
├── General
│   ├── date = 2017-07-13
│   ├── original_filename = Pt-EDX04.bcf
│   ├── time = 11:18:16
│   └── title = Original Data Wedge
├── Sample
│   ├── elements = ['Cu', 'Ga', 'Mo', 'Pt']
│   ├── name = Undefinded
│   └── xray_lines = ['Cu_Ka', 'Ga_Ka', 'Mo_Ka', 'Pt_La']
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_TEM

Adjust all the metadata for the microscope parameters.

(Live_time is not needed for quantification but currently prevents the code from running further if not set.)

In [42]:
#Pt_wedge.set_microscope_parameters(beam_energy = 200)
Pt_wedge.set_microscope_parameters(live_time = 0.002696) #dwell time per pixel
Pt_wedge.set_microscope_parameters(beam_current = 0.039) #in nA

Adjust the metadata for all the sample parameters.

In [43]:
Pt_wedge.set_elements(['Pt'])
Pt_wedge.set_lines(['Pt_La'])

Finally look at the original raw data in plot mode.

In [44]:
Pt_wedge.plot(True)

Save the data, with all the added metadata, for repeat processing if necessary.

In [45]:
Pt_wedge.save('Pt01EDX-processed')

## 3. Extracting the intensity map for wedge 1

Define the background subtraction windows:

*These can be edited or modified manually as necessary depending on what they look like.*

In [46]:
Pt_wedge.add_lines()
bw_Pt = Pt_wedge.estimate_background_windows(line_width=[5.0, 2.0])
bw_Pt

array([[ 8.47913271,  8.63962726,  9.7630891 ,  9.92358364]])

Define the integration windows:

*The automatic integration window is 2xFWHM. However this might vary depending on counts. For accurate quantification we want to integrate over the same absolute windows so we need to define them for exact use later on. Using FWHM is a reasable first measure approach for this.*

In [47]:
iw_Pt =  Pt_wedge.estimate_integration_windows(windows_width=2)
iw_Pt[0][1] = 9.65
iw_Pt

[[9.281605451805987, 9.65]]

Plot the sum spectrum with all the background windows to check that nothing overlaps detrimentally. 

*The integration windows should only contain the individual element peak in question otherwise some form of deconvolution method may be required for integration. The background windows should overlap with no peaks.*

In [48]:
Pt_wedge.sum().plot(True, background_windows=bw_Pt, integration_windows=iw_Pt)

Now extract the intensity map from your spectrum image.

In [49]:
inten_Pt = Pt_wedge.get_lines_intensity(integration_windows=iw_Pt, background_windows=bw_Pt)
inten_Pt[0].plot()
inten_Pt

[<BaseSignal, title: X-ray line intensity of Original Data Wedge: Pt_La at 9.44 keV, dimensions: (1024, 64|)>]

This final line is needed when combining multiple x-ray lines for each element.

## 4. Repeat Steps 2 & 3 for Wedge 2 (Ni in this example)

In [68]:
Ni_wedge = hs.load('EDX-Data/Ni-EDX04.bcf')
#Separate out the image and spectrum image from .bcl files.
Ni_wedge_image = Ni_wedge[0]
Ni_wedge = Ni_wedge[1]

Ni_wedge.set_signal_type('EDS_TEM')
Ni_wedge.metadata.General.title = 'Original Data Wedge'

Ni_wedge

c:\users\macark\documents\github\hyperspy\hyperspy\io_plugins\bcf.py:829: RuntimeWarning: overflow encountered in long_scalars
  real_time = line_cnt_sum * line_avg * pix_avg * pix_time * width / 1000000.0


<EDSTEMSpectrum, title: Original Data Wedge, dimensions: (1024, 1024|4096)>

In [57]:
Ni_wedge.axes_manager

<Axes manager, axes: (1024, 1024|4096)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
           width |   1024 |      0 |       0 |      12 |     nm 
          height |   1024 |      0 |       0 |      12 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   4096 |        |   -0.95 |    0.01 |    keV

In [55]:
Ni_wedge.axes_manager.gui()

In [58]:
Ni_wedge.set_elements(['Ni'])
Ni_wedge.set_lines(['Ni_Ka', 'Ni_Kb'])
Ni_wedge.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 45.0
│       │       ├── detector_type = SuperX
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       └── real_time = -1345.724416
│       ├── Stage
│       │   └── tilt_alpha = 0.0
│       ├── beam_energy = 200
│       └── magnification = 7100
├── General
│   ├── date = 2017-07-13
│   ├── original_filename = Ni-EDX04.bcf
│   ├── time = 13:18:17
│   └── title = Original Data Wedge
├── Sample
│   ├── elements = ['Ni']
│   ├── name = Undefinded
│   └── xray_lines = ['Ni_Ka', 'Ni_Kb']
└── Signal
    ├── binned = True
    ├── quantity = X-rays (Counts)
    └── signal_type = EDS_TEM

In [59]:
Ni_wedge.set_microscope_parameters(beam_energy = 200)
Ni_wedge.set_microscope_parameters(live_time = 0.002567)
Ni_wedge.set_microscope_parameters(beam_current = 0.074) #in nA
Ni_wedge.set_microscope_parameters(real_time = 0.002567) #dwell time per pixel

In [73]:
Ni_wedge.plot(True)
Ni_wedge.add_lines()
bw_Ni = Ni_wedge.estimate_background_windows(line_width=[5.0, 2.0])
bw_Ni

c:\program files\anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


array([[ 6.61175942,  6.75614952,  9.56970943,  9.72871415],
       [ 6.61175942,  6.75614952,  9.56970943,  9.72871415],
       [ 6.61175942,  6.75614952,  8.34628116,  8.49552173]])

In [74]:
iw_Ni = Ni_wedge.estimate_integration_windows(windows_width=2)
iw_Ni

[[7.898559422407979, 8.197040577592022],
 [9.092695283088833, 9.410704716911166],
 [7.333709903386694, 7.622490096613307]]

In [75]:
Ni_wedge.sum().plot(True, background_windows=bw_Ni, integration_windows=iw_Ni)

c:\program files\anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [70]:
Ni_wedge.inav[0:1024, 430:494].plot()

c:\program files\anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
c:\program files\anaconda3\lib\site-packages\matplotlib\axes\_base.py:3193: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0, top=0
  'bottom=%s, top=%s') % (bottom, top))


In [71]:
Ni_wedge0 = Ni_wedge
Ni_wedge = Ni_wedge.inav[0:1024, 430:494]
Ni_wedge.plot()

c:\program files\anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [34]:
Ni_wedge.save('Ni02EDX-processed')

In [76]:
inten_Ni = Ni_wedge.get_lines_intensity(background_windows=bw_Ni)
inten_Ni

[<BaseSignal, title: X-ray line intensity of Original Data Wedge: Cu_Ka at 8.05 keV, dimensions: (1024, 64|)>,
 <BaseSignal, title: X-ray line intensity of Original Data Wedge: Ga_Ka at 9.25 keV, dimensions: (1024, 64|)>,
 <BaseSignal, title: X-ray line intensity of Original Data Wedge: Ni_Ka at 7.48 keV, dimensions: (1024, 64|)>]

## 5. Conversion from the Intensity Maps to Cross Sections

The following function will calculate the cross section of a for a single element wedge sample with a known angle. It assumes that the line profile is parallel to the x-axis in the image. 

*The function will also print the error in the gradient measurement which is important for subsequent error analysis of the quantificationt results.*

In [77]:
def cross_section_from_wedge(intensity,
                            angle,
                            atomic_density):
    """
    
    Parameters
    ----------
        
    intensity : array
        
    angle : float
        Of the FIB wedge in degrees.
    atomic_density: float 
        Of the particular element in units of atoms/nm^3.
        (Pt = 33.2026, Ni = 45.8566)
    
    Returns
    -------
    cross_section : float
        partial ionisation cross section
    
    """
    
    from scipy import constants
    import math
    e = constants.e
    parameters = intensity[0].metadata.Acquisition_instrument.TEM
    curr = parameters.beam_current
    dwell = parameters.Detector.EDS.real_time
    width = intensity[0].axes_manager[0].scale
    angle = angle/360*constants.pi*2

    #begin by converting the intensity array to a line profile.
    #works for square images, needs checking for those which are non-square.
    line_scan = np.zeros(intensity[0].axes_manager.signal_shape[0])
    line_scan.astype(float)
    for row in range(intensity[0].axes_manager.signal_shape[1]):
        for col in range(intensity[0].axes_manager.signal_shape[0]):
            line_scan[col] = intensity[0].data[row, col] + line_scan[col]
    
    line_scan = line_scan/(intensity[0].axes_manager.signal_shape[1]*width)
    x_data = np.zeros(intensity[0].axes_manager.signal_shape[0])
    x_data.astype(float)
    for i in range(intensity[0].axes_manager.signal_shape[0]):
        x_data[i] = i*width
    
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6), dpi=80)
    plt.plot(x_data, line_scan)
    plt.show()
    #curve fitting to line scan to find gradient
    #using  a non-linear least squares fitting.
    from scipy.optimize import curve_fit
    def func(x, m, c):
        return m*x + c
    popt, pcov = curve_fit(func, x_data, line_scan)
    
    m = popt[0]
    print('m =', m)
    perr = np.sqrt(np.diag(pcov))
    m_err = perr[0] #the error in m measurement
    print('Error in m =', m_err)
    
    #calculate cross_section
    cross_section = (e*1E19*m)/(curr*dwell*atomic_density*math.tan(angle))
    return cross_section

Calculate the cross section for each element from the previously extracted intensity maps.

In [79]:
inten_Pt

[<BaseSignal, title: X-ray line intensity of Original Data Wedge: Pt_La at 9.44 keV, dimensions: (1024, 64|)>]

In [97]:
inten_Pt[0].save('PtEDX01-inten')

In [80]:
cross_section_Pt = cross_section_from_wedge(inten_Pt,
                                            0.269,
                                            33.2026)

AttributeError: 'DictionaryTreeBrowser' object has no attribute 'beam_current'

An interative process of cropping the line profile to only the linear region is needed for accurate cross section determination. We only want the gradient where the x-ray generation is linear with sample thickness.

In [80]:
inten_Ni[0] = inten_Ni[0].isig[:400, :]
inten_Ni

[<Signal2D, title: X-ray line intensity of Original Data Wedge: Ni_Ka at 7.48 keV, dimensions: (|400, 64)>]

In [81]:
cross_section_Ni = cross_section_from_wedge(inten_Ni,
                                           0.401,
                                           45.8566)

m = 0.000323058447471
Error in m = 2.00357066389e-06


//anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [86]:
print('Cross Section Pt', cross_section_Pt)
print('Cross Section Ni', cross_section_Ni)

Cross Section Pt 7.81476090922
Cross Section Ni 8.48991263031


In [98]:
inten_Ni[0].save('NiEDX02-inten')

## 6.Storing the Cross Sections and Windows into a Library for Later Use

Convert the cross section and windows into a dictionary format.

In [100]:
calibration_Pt = dict = {'lines': 'PtLa', 'cross_section': cross_section_Pt,
                         'Background windows': bw_Pt, 'Integration windows': iw_Pt}
calibration_Ni = dict = {'lines': 'NiKa', 'cross_section': cross_section_Ni, 
                         'Background windows': bw_Ni, 'Integration windows': iw_Ni}

In [101]:
calibration_Pt

{'Background windows': array([[ 8.47913271,  8.63962726,  9.7630891 ,  9.92358364]]),
 'Integration windows': [[9.281605451805987, 9.65]],
 'cross_section': 7.8147609092183261,
 'lines': 'PLa'}

In [102]:
calibration = dict = {'Ni': calibration_Ni, 'Pt': calibration_Pt}
calibration['Ni']['cross_section']

8.4899126303055183

In [103]:
cal = hs.signals.BaseSignal([0])
cal.metadata.calibration = calibration
cal.save('ChemiSTEMCal_V01')

Overwrite 'ChemiSTEMCal_V01.hdf5' (y/n)?
y


The following function can be used to check which elements are stored in the dictionary.

In [104]:
calibration.keys()

dict_keys(['Ni', 'Pt'])

**Any functions below this point are for testing, and not needed in the method above.**

In [92]:
calibration['Ni'] = calibration_Ni

In [93]:
del calibration['Ni']

In [94]:
calibration2 = cal.metadata.calibration